In [1]:
# !pip install torch==2.2
!pip install torch
!pip install torchvision
!pip install seaborn
!pip install numpy
!pip install matplotlib
# !pip install torchsummary
# !pip install torchview
# !pip install graphviz
# !pip install torchviz
!pip install pandas
# !pip install efficientnet_pytorch

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Subset
from torch.utils.data import random_split
import seaborn as sns
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
BATCH_SIZE = 64
num_epoch = 10
learning_rate = 1e-4
class_size = 10

In [7]:
transform_train = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.RandomHorizontalFlip(p=0.7)])

transform_test = transforms.Compose([transforms.Resize((224, 224))])

In [8]:
!ls /content/drive/MyDrive/hurricane_damage/

QNN_TRAIN_CPU_hurricane_W4_X_S0_E10000_s1.npy	  QNN_TRAIN_CPU_hurricane_W4_XY_S0_E5000_s1.npy
QNN_TRAIN_CPU_hurricane_W4_X_S0_E10000_s2.npy	  QNN_TRAIN_CPU_hurricane_W4_XY_S5000_E10000_s1.npy
QNN_TRAIN_CPU_hurricane_W4_X_S0_E5000_s1.npy	  raw_hurricane_dataset
QNN_TRAIN_CPU_hurricane_W4_X_S5000_E10000_s1.npy  train_another_labels.npy
QNN_TRAIN_CPU_hurricane_W4_XY_S0_E10000_s1.npy


In [9]:
q_train_images = np.load("/content/drive/MyDrive/hurricane_damage/QNN_TRAIN_CPU_hurricane_W4_XY_S0_E10000_s1.npy")


In [10]:
print(f'shape of raw quantum train data: {q_train_images.shape}')
train_output_shape = (4 * 10000, 127, 127, 3)
q_train_images = np.transpose(q_train_images, (4, 0, 1, 2, 3))
q_train_images = np.reshape(q_train_images, train_output_shape)
q_train_images = np.transpose(q_train_images, (0, 3, 1, 2))
print(f'shape of quantum train data: {q_train_images.shape}')

shape of raw quantum train data: (10000, 127, 127, 3, 4)
shape of quantum train data: (40000, 3, 127, 127)


In [11]:
torch.manual_seed(2024)

train_ref = torchvision.datasets.ImageFolder("/content/drive/MyDrive/hurricane_damage/raw_hurricane_dataset/train_another/", transform=transforms.ToTensor())

train_loader_ref = DataLoader(train_ref, batch_size=BATCH_SIZE, shuffle=False)

In [12]:
labels = np.load("/content/drive/MyDrive/hurricane_damage/train_another_labels.npy")
print(labels.shape)

(10000,)


In [13]:
transformed_labels = torch.Tensor(labels.tolist() * 4).to(torch.int64)
print(transformed_labels.shape)

torch.Size([40000])


In [14]:
train_dataset = TensorDataset(torch.Tensor(q_train_images), transformed_labels)

val_size = 8000

train_size = len(train_dataset) - val_size

train, val = random_split(train_dataset, [train_size, val_size])
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(val, batch_size=BATCH_SIZE, shuffle=False)

In [15]:
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

for images, labels in train_loader:
    print('shape: ', images.shape)
    print(labels.tolist())
    plt.figure(figsize=(100, 100))
    plt.axis('off')
    plt.imshow(make_grid(images, nrow=16).permute((1, 2, 0)))
    break

Output hidden; open in https://colab.research.google.com to view.

In [18]:
from torchvision import models

model = models.efficientnet_v2_s(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 171MB/s]


In [19]:
print(model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [22]:
input_last_year = model.classifier[1].in_features
model.classifier[1] = nn.Linear(input_last_year, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
epoch_loss_history = []
epoch_acc_history = []

In [24]:

# torch.cuda.empty_cache()

for epoch in range(num_epoch):
    epoch_avg_loss = 0
    for i, (imgs, labels) in enumerate(train_loader):
        imgs = transform_train(imgs.to(device))
        labels = labels.to(device)

        labels_hat = model(imgs)
        n_corrects = (labels_hat.argmax(axis=1) == labels).sum().item()
        loss_value = criterion(labels_hat, labels)
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()

        epoch_avg_loss += loss_value.item()

        accuracy = (n_corrects / labels.size(0)) * 100

        if (i+1) % 10 == 0:
            print(f'epoch {epoch + 1}/{num_epoch}, step: {i + 1}/{train_size/BATCH_SIZE}: loss = {loss_value:.5f}, acc = {accuracy:.2f}')

    epoch_loss_history.append(epoch_avg_loss / len(train_loader))

    with torch.no_grad():
        correct = 0
        samples = 0

        for i, (images, labels) in enumerate(test_loader):
            images = transform_test(images.to(device))
            labels = labels.to(device)
            outputs = model(images)
            _, predictions = outputs.max(1)
            correct += (predictions == labels).sum()
            samples += predictions.size(0)

        accuracy = float(correct) / float(samples) * 100
        epoch_acc_history.append(accuracy)

        print(f"epoch {epoch + 1}/{num_epoch}, val accuracy = {accuracy:.2f}%. Correct {correct} out of {samples} samples")

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


epoch 1/10, step: 10/500.0: loss = 0.53340, acc = 82.81
epoch 1/10, step: 20/500.0: loss = 0.29123, acc = 95.31
epoch 1/10, step: 30/500.0: loss = 0.20126, acc = 90.62
epoch 1/10, step: 40/500.0: loss = 0.15045, acc = 92.19
epoch 1/10, step: 50/500.0: loss = 0.11374, acc = 96.88
epoch 1/10, step: 60/500.0: loss = 0.12089, acc = 96.88
epoch 1/10, step: 70/500.0: loss = 0.09026, acc = 96.88
epoch 1/10, step: 80/500.0: loss = 0.14775, acc = 93.75
epoch 1/10, step: 90/500.0: loss = 0.08194, acc = 95.31
epoch 1/10, step: 100/500.0: loss = 0.03464, acc = 98.44
epoch 1/10, step: 110/500.0: loss = 0.07404, acc = 96.88
epoch 1/10, step: 120/500.0: loss = 0.03462, acc = 100.00
epoch 1/10, step: 130/500.0: loss = 0.04614, acc = 98.44
epoch 1/10, step: 140/500.0: loss = 0.08070, acc = 98.44
epoch 1/10, step: 150/500.0: loss = 0.09168, acc = 95.31
epoch 1/10, step: 160/500.0: loss = 0.06464, acc = 98.44
epoch 1/10, step: 170/500.0: loss = 0.01900, acc = 100.00
epoch 1/10, step: 180/500.0: loss = 0.

In [25]:
print(epoch_loss_history)


[0.09181079315766692, 0.013283439244900365, 0.008672975107532692, 0.007014932877420506, 0.005685827836548924, 0.0036459875853506675, 0.0032127326549361897, 0.005143991610404555, 0.003569534302269858, 0.003389765918469493]


In [26]:
print(epoch_acc_history)


[98.32499999999999, 99.2875, 99.375, 99.375, 99.6625, 98.6125, 99.4375, 99.3, 99.55000000000001, 99.5]


In [27]:
with open('/content/drive/MyDrive/res/history/effnet_hurricane_W4_xy_s1_loss_history.txt', "w") as f:
    for item in epoch_loss_history:
        f.write(f'{item:.5f}\n')

In [28]:
with open('/content/drive/MyDrive/res/history/effnet_hurricane_W4_xy_s1_acc_history.txt', "w") as f:
    for item in epoch_acc_history:
        f.write(f'{item:.5f}\n')

In [30]:
torch.save(model, '/content/drive/MyDrive/res/effnet_hurricane_W4_xy_s1.pth')